## DEMO 5: ENTRENAMIENTO DEL MODELO DE IA

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pickle
import torch   

### CARGA Y PREPROCESAMIENTO DE DATOS

In [2]:

df = pd.read_csv('T_C03-1C2025_historico_prod.csv', sep=';')         # carga del archivo CSV

df = pd.get_dummies(df, dtype='int')                                # one hot encoding

df = df.drop(columns=['año'])                                       # elimino la columna año

y = df['ventas_und']                                                # separo y

X = df.drop(columns=['ventas_und'])                                 # separo X  "X.columns.values.tolist()"

### ENTRENAMIENTO DEL MODELO

In [3]:
# escalo los datos

e = StandardScaler()
X = e.fit_transform(X.values)

# divido el dataset en entrenamiento

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=9)

# convierto mis arrays en tensores

X_train_t = torch.tensor(X_train).float()
X_test_t = torch.tensor(X_test).float()
y_train_t = torch.tensor(y_train.values).float()
y_test_t = torch.tensor(y_test.values).float()
y_test_t = y_test_t[:, None]
y_train_t = y_train_t[:, None]

# definimos e instanciamos el modelo de la red neuronal

model = torch.nn.Sequential(
    torch.nn.Linear(47, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 50),
    torch.nn.ReLU(),
    torch.nn.Linear(50, 30),
    torch.nn.ReLU(),
    torch.nn.Linear(30, 1),
)

# defino los hiperparametros de entrenamiento:

lr = 0.001
epochs = 50000
e_print = 100
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

# realizo el entrenamiento (dependiendo el procesador esta operacion tarda unos 6 minutos):

for epoch in range(1, epochs+1):
    y_pred = model(X_train_t)
    loss = loss_fn(y_pred, y_train_t)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if epoch % e_print == 0:
        print(f"epoch {epoch} \t Loss: {round(loss.item(),4)}")

epoch 100 	 Loss: 549838208.0
epoch 200 	 Loss: 291998464.0
epoch 300 	 Loss: 105895560.0
epoch 400 	 Loss: 70174440.0
epoch 500 	 Loss: 51397540.0
epoch 600 	 Loss: 45489040.0
epoch 700 	 Loss: 43806060.0
epoch 800 	 Loss: 42710388.0
epoch 900 	 Loss: 41686832.0
epoch 1000 	 Loss: 40734312.0
epoch 1100 	 Loss: 39845364.0
epoch 1200 	 Loss: 38992072.0
epoch 1300 	 Loss: 38141608.0
epoch 1400 	 Loss: 37284920.0
epoch 1500 	 Loss: 36413320.0
epoch 1600 	 Loss: 35514092.0
epoch 1700 	 Loss: 34593020.0
epoch 1800 	 Loss: 33657584.0
epoch 1900 	 Loss: 32679002.0
epoch 2000 	 Loss: 31642654.0
epoch 2100 	 Loss: 30604196.0
epoch 2200 	 Loss: 29530612.0
epoch 2300 	 Loss: 28461244.0
epoch 2400 	 Loss: 27337742.0
epoch 2500 	 Loss: 26255064.0
epoch 2600 	 Loss: 25186464.0
epoch 2700 	 Loss: 24175954.0
epoch 2800 	 Loss: 22961268.0
epoch 2900 	 Loss: 21062174.0
epoch 3000 	 Loss: 19384950.0
epoch 3100 	 Loss: 17643590.0
epoch 3200 	 Loss: 15680605.0
epoch 3300 	 Loss: 13905858.0
epoch 3400 	 Los

### SE GUARDAN LOS PARÁMETROS ENTRENABLES DEL MODELO

In [5]:
torch.save(model.state_dict(), 'T_C03-1C2025_modelo.pth')

# guarda los datos del para reescalar
with open('T_C03-1C2025_scaler.pkl','wb') as f:
    pickle.dump(e, f)